In [ ]:
%load_ext autoreload
%autoreload 2
    
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import random

from scipy.optimize import curve_fit
from scipy.stats import rayleigh, rice, shapiro, f_oneway
from scipy.special import i0

from functionsFIMS import getAnalysisNumbers, plotDataSets
from runDataClass import runData
from gasDataClass import gasData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(1148)
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    binWidth=3
    simData.plotAvalancheSize(binWidth)
    simData.plotAvalancheFits(binWidth)

    thresh = 10
    fitPolya = myPolya(24.701, 0.488)
    efficiency = fitPolya.calcEfficiencyErrs(thresh, 0.671, 0.095)
    print(f'{efficiency[0]:.3f} (+{efficiency[1]:.3f}, {efficiency[2]:.3f})')

In [ ]:
if __name__ == '__main__':
    randInt = random.randint(0, 999)
    simData.plotAvalanche2D(randInt, plotName='Random')
    simData.plotAvalancheSignal(randInt)
    simData.plotSignalvsGain()

In [ ]:
TESTDATA = runData(1172)

In [ ]:
def getLargestRadius(xData, yData):
    """
    TODO
    """
    radius2 = xData**2 + yData**2
    maxRadius = math.sqrt(radius2.max())
    minRadius = math.sqrt(radius2.min())

    return maxRadius, minRadius

    

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.', alpha=.15
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', marker='.', alpha=0.15
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    padLength = runData.getRunParameter('Pad Length')
    pitch = runData.getRunParameter('Pitch')
    axLim = pitch/math.sqrt(3)
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)

In [ ]:
def showIonHist(runData):
    """
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    posIons = posIons[posIons['Initial z']!=posIons['Initial z'].iloc[0]]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]

    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad,
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad,
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad,
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    nominalFieldBundle = plt.Circle(
            (0, 0), runData.getRunParameter('Field Bundle Radius'), 
            facecolor='none', edgecolor='c', lw=2
        )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('All Ion Initial Locations')
    ax = fig.add_subplot(111)
    '''
    ax.hist2d(
        posIons['Initial x'], posIons['Initial y'],
        bins=101, cmin=1
    )
    '''
    '''
    ax.hist2d(
        bfIons['Initial x'], bfIons['Initial y'],
        bins=101, cmin=1
    )
    '''
    ax.hist2d(
        capIons['Initial x'], capIons['Initial y'],
        bins=101, cmin=1
    )

    #ax.add_patch(bfCircle)
    #ax.add_patch(capCircle0)
    #ax.add_patch(capCircle1)
    ax.add_patch(nominalFieldBundle)
    
    runData._plotAddCellGeometry(ax, 'xy')

    padLength = runData.getRunParameter('Pad Length')
    pitch = runData.getRunParameter('Pitch')
    
    largeLim = pitch/math.sqrt(3)
    axLim = padLength
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    #ax.legend()

    return fig

_ = showIonHist(TESTDATA)

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.'
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', marker='.'
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    #axLim = runData.getRunParameter('Pad Length')
    #ax.set_xlim(-axLim, axLim)
    #ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)

In [ ]:
def getEffData(efficiencyRuns):
    allRunData = []

    runParams = [
        'Electric Field Ratio'
    ]
    calcParams = [
        'Raw Gain',
        'Trimmed Gain',
        'Gain Error',
        'Raw Efficiency (10e)',
        'Raw Efficiency Error',
        'Efficiency (10e)',
        'Efficiency Error',
        'Average IBN',
        'IBN Error',
        'Average IBF',
        'IBF Error'
    ]
    
    for inRun in efficiencyRuns:
        inRunData = runData(inRun)
    
        thisRunData = {'runNumber': inRun}
        
        # Loop over keys to populate the dictionary
        for inParam in runParams:
            thisRunData[inParam] = inRunData.getRunParameter(inParam)
        for inParam in calcParams:
            thisRunData[inParam] = inRunData.getCalcParameter(inParam)

        allRunData.append(thisRunData)
    
    return pd.DataFrame(allRunData)

In [ ]:
#efficiencyRuns = np.arange(1200, 1205)#Old geometry
#efficiencyRuns = np.arange(1205, 1210)#updated initial geometry
#efficiencyRuns = np.arange(1212, 1218)#T2K gas
T2K_OLD = getEffData(np.arange(1221, 1232))#T2K gas old FEM (r=55)
T2K_NEW = getEffData(np.arange(1233, 1243))#T2K gas (r=65)
T2K_TEST = getEffData(np.arange(1245, 1255))#T2K gas (r=75)

In [ ]:
def myExpo(x, a, b, c):
    return a*np.exp(b*(x-c))

def mySigmoid(x, k, x0):
    return 1/(1+np.exp(-k*(x-x0)))

def plotGainAndEfficiency(simData):
    fitFields = np.arange(min(simData['Electric Field Ratio'])-2, max(simData['Electric Field Ratio'])+2)
    
    l0 = [5, 0.05, 50]
    lopt, lcov = curve_fit(
        myExpo, simData['Electric Field Ratio'], simData['Trimmed Gain'], 
        l0, method='lm', sigma=simData['Gain Error'], absolute_sigma=True
    )
    fitExpo = myExpo(fitFields, *lopt)
    
    
    p0 = [0.5, 40] 
    popt, pcov = curve_fit(
        mySigmoid, simData['Electric Field Ratio'], simData['Efficiency (10e)'], 
        p0, method='lm', sigma=simData['Efficiency Error'], absolute_sigma=True
    )
    fitSigmoid = mySigmoid(fitFields, *popt)
    
    print(f'Expo fit parameters: {lopt[0]:.3f}, {lopt[1]:.3f}, {lopt[2]:.3f}')
    print(f'Sigmoid fit parameters: {popt[0]:.3f}, {popt[1]:.3f}')
    
    fig = plt.figure(figsize=(10, 4))
    fig.suptitle('FIMS Simulations')
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.errorbar(
        simData['Electric Field Ratio'], simData['Trimmed Gain'], 
        yerr=simData['Gain Error'], ls='', marker='x', label='Gain'
    )
    ax1.plot(fitFields, fitExpo, label='Exponential Fit', c='r', ls='--')
    ax1.axhline(y=10, label='10-electron Threshold', c='g', ls=':')
    
    ax2.errorbar(
        simData['Electric Field Ratio'], simData['Efficiency (10e)'], 
        yerr=simData['Efficiency Error'], ls='', marker='x', label='Efficiency'
    )
    ax2.plot(fitFields, fitSigmoid, label='Sigmoid Fit', c='r', ls='--')
    ax2.axhline(y=.95, label='95% Target', c='g', ls=':')
    
        
    ax1.set_title('Gain')
    ax1.set_xlabel('Field Ratio')
    ax1.set_ylabel('Electron Gain')
    ax1.set_yscale('log')
    ax1.legend()
    ax1.grid()
    
    ax2.set_title('Efficiency')
    ax2.set_xlabel('Field Ratio')
    ax2.set_ylabel('10-Electron Efficiency')
    ax2.legend()
    ax2.grid()
    
    plt.tight_layout()
    plt.show()


In [ ]:
plotGainAndEfficiency(T2K_NEW)

In [ ]:
def plotIBNandIBF(simData):
    fig = plt.figure(figsize=(10, 4))
    fig.suptitle('FIMS Simulations')
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    
    ax1.errorbar(
        simData['Electric Field Ratio'], simData['Average IBN'], 
        yerr=simData['IBN Error'], ls='', marker='x', label='IBN'
    )
    
    
    ax2.errorbar(
        simData['Electric Field Ratio'], simData['Average IBF'], 
        yerr=simData['IBF Error'], ls='', marker='x', label='IBF'
    )
    ax2.axhline(
        y=np.average(simData['Average IBF'], 
        weights=1/simData['IBF Error']**2), ls='--'
    )
    
    
    ax1.set_title('IBN')
    ax1.set_xlabel('Field Ratio')
    ax1.set_ylabel('Average IBN')
    ax1.set_yscale('log')
    ax1.legend()
    ax1.grid()
    
    ax2.set_title('IBF')
    ax2.set_xlabel('Field Ratio')
    ax2.set_ylabel('Average IBF')
    ax2.legend()
    ax2.grid()

    plt.tight_layout()
    plt.show()

In [ ]:
plotIBNandIBF(T2K_NEW)

In [ ]:
fitFields = np.arange(min(T2K_OLD['Electric Field Ratio'])-2, max(T2K_OLD['Electric Field Ratio'])+2)

l0 = [5, 0.05, 50]
OLD_lopt, OLD_lcov = curve_fit(
    myExpo, T2K_OLD['Electric Field Ratio'], T2K_OLD['Trimmed Gain'], 
    l0, method='lm', sigma=T2K_OLD['Gain Error'], absolute_sigma=True
)
NEW_lopt, NEW_lcov = curve_fit(
    myExpo, T2K_NEW['Electric Field Ratio'], T2K_NEW['Trimmed Gain'], 
    l0, method='lm', sigma=T2K_NEW['Gain Error'], absolute_sigma=True
)
TEST_lopt, TEST_lcov = curve_fit(
    myExpo, T2K_TEST['Electric Field Ratio'], T2K_TEST['Trimmed Gain'], 
    l0, method='lm', sigma=T2K_TEST['Gain Error'], absolute_sigma=True
)
OLDExpo = myExpo(fitFields, *OLD_lopt)
NEWExpo = myExpo(fitFields, *NEW_lopt)
TESTExpo = myExpo(fitFields, *TEST_lopt)


p0 = [0.5, 40]
OLD_popt, OLD_pcov = curve_fit(
    mySigmoid, T2K_OLD['Electric Field Ratio'], T2K_OLD['Efficiency (10e)'], 
    p0, method='lm', sigma=T2K_OLD['Efficiency Error'], absolute_sigma=True
)
NEW_popt, NEW_pcov = curve_fit(
    mySigmoid, T2K_NEW['Electric Field Ratio'], T2K_NEW['Efficiency (10e)'], 
    p0, method='lm', sigma=T2K_NEW['Efficiency Error'], absolute_sigma=True
)
TEST_popt, TEST_pcov = curve_fit(
    mySigmoid, T2K_TEST['Electric Field Ratio'], T2K_TEST['Efficiency (10e)'], 
    p0, method='lm', sigma=T2K_TEST['Efficiency Error'], absolute_sigma=True
)
OLDSigmoid = mySigmoid(fitFields, *OLD_popt)
NEWSigmoid = mySigmoid(fitFields, *NEW_popt)
TESTSigmoid = mySigmoid(fitFields, *TEST_popt)



fig = plt.figure(figsize=(10, 4))
fig.suptitle('FIMS Simulations')
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


ax1.errorbar(
    T2K_OLD['Electric Field Ratio'], T2K_OLD['Trimmed Gain'], 
    yerr=T2K_OLD['Gain Error'], c='r', ls='', marker='x', label='Hole Radius = 55um'
)
ax1.errorbar(
    T2K_NEW['Electric Field Ratio'], T2K_NEW['Trimmed Gain'], 
    yerr=T2K_NEW['Gain Error'], c='g', ls='', marker='x', label='Hole Radius = 65um'
)
ax1.errorbar(
    T2K_TEST['Electric Field Ratio'], T2K_TEST['Trimmed Gain'], 
    yerr=T2K_TEST['Gain Error'], c='b', ls='', marker='x', label='Hole Radius = 75um'
)
ax1.plot(fitFields, OLDExpo, c='r', ls='--')
ax1.plot(fitFields, NEWExpo, c='g', ls='--')
ax1.plot(fitFields, TESTExpo, c='b', ls='--')
ax1.axhline(y=10, label='10-electron Threshold', c='m', ls='--')

ax2.errorbar(
    T2K_OLD['Electric Field Ratio'], T2K_OLD['Efficiency (10e)'], 
    yerr=T2K_OLD['Efficiency Error'], c='r', ls='', marker='x', label='Hole Radius = 55um'
)
ax2.errorbar(
    T2K_NEW['Electric Field Ratio'], T2K_NEW['Efficiency (10e)'], 
    yerr=T2K_NEW['Efficiency Error'], c='g', ls='', marker='x', label='Hole Radius = 65um'
)
ax2.errorbar(
    T2K_TEST['Electric Field Ratio'], T2K_TEST['Efficiency (10e)'], 
    yerr=T2K_TEST['Efficiency Error'], c='b', ls='', marker='x', label='Hole Radius = 75um'
)

ax2.plot(fitFields, OLDSigmoid, c='r', ls='--')
ax2.plot(fitFields, NEWSigmoid, c='g', ls='--')
ax2.plot(fitFields, TESTSigmoid, c='b', ls='--')
ax2.axhline(y=.95, label='95% Target', c='m', ls='--')

ax1.axvline(x=64, label='Transparency Limit (55um)', c='r', ls=':')
ax2.axvline(x=64, label='Transparency Limit (55um)', c='r', ls=':')

ax1.set_title('Gain')
ax1.set_xlabel('Field Ratio')
ax1.set_ylabel('Electron Gain')
ax1.set_yscale('log')
ax1.legend()
ax1.grid()

ax2.set_title('Efficiency')
ax2.set_xlabel('Field Ratio')
ax2.set_ylabel('10-Electron Efficiency')
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()

In [ ]:
def plotSimulationData(plotData):
    """
    TODO
    """
    fig, axes = plt.subplots(2, 2, figsize=(10, 8))
    fig.suptitle('FIMS Simulations')
    
    allAxes = axes.flatten()
    for inData in plotData:
        
    
        allAxes[0].errorbar(
            inData['simData']['Electric Field Ratio'], inData['simData']['Trimmed Gain'], 
            yerr=inData['simData']['Gain Error'], 
            c=inData['c'], ls='', marker='x', 
            label=f'Hole Radius = {inData['radius']}um'
        )
        
        
        allAxes[1].errorbar(
            inData['simData']['Trimmed Gain'], inData['simData']['Efficiency (10e)'], 
            xerr=inData['simData']['Gain Error'], yerr=inData['simData']['Efficiency Error'], 
            c=inData['c'], ls='', marker='x', 
            label=f'{inData['radius']}um'
        )
    
        allAxes[2].errorbar(
            inData['simData']['Trimmed Gain'], inData['simData']['Average IBN'], 
            xerr=inData['simData']['Gain Error'], yerr=inData['simData']['IBN Error'], 
            c=inData['c'], ls='', marker='x', 
            label=f'{inData['radius']}um'
        )
        
        
        allAxes[3].errorbar(
            inData['simData']['Trimmed Gain'], inData['simData']['Average IBF'], 
            xerr=inData['simData']['Gain Error'], yerr=inData['simData']['IBF Error'], 
            c=inData['c'], ls='', marker='x', 
            label=f'{inData['radius']}um'
        )
        allAxes[3].axhline(
            y=np.average(inData['simData']['Average IBF'], weights=1/inData['simData']['IBF Error']**2), 
            c=inData['c'], ls='--'
        )
    
    
    allAxes[0].axhline(y=10, label='10-electron Threshold', c='m', ls='--')
    allAxes[0].axvline(x=64, label='Transparency Limit (55um)', c='r', ls=':')
    allAxes[1].axhline(y=.95, label='95% Target', c='m', ls=':')
    
    
    gainLim = [4, 4e2]
    
    allAxes[0].set_title('Gain')
    allAxes[0].set_xlabel('Field Ratio')
    allAxes[0].set_ylabel('Electron Gain')
    allAxes[0].set_yscale('log')
    allAxes[0].set_ylim(gainLim)
    
    allAxes[1].set_title('Efficiency')
    allAxes[1].set_xlabel('Electron Gain')
    allAxes[1].set_ylabel('10-Electron Efficiency')
    allAxes[1].set_xscale('log')
    allAxes[1].set_xlim(gainLim)
    
    allAxes[2].set_title('IBN')
    allAxes[2].set_xlabel('Electron Gain')
    allAxes[2].set_ylabel('Average IBN')
    allAxes[2].set_xscale('log')
    allAxes[2].set_yscale('log')
    allAxes[2].set_xlim(gainLim)
    allAxes[2].set_ylim([1, 1e2])
    
    allAxes[3].set_title('IBF')
    allAxes[3].set_xlabel('Electron Gain')
    allAxes[3].set_ylabel('Average IBF')
    allAxes[3].set_xscale('log')
    allAxes[3].set_xlim(gainLim)
    
    
    for inAx in allAxes:
        inAx.grid()
        inAx.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
plotData = [
    {
        'simData': T2K_OLD,
        'radius': 55,
        'c': 'r'
    },
    {
        'simData': T2K_NEW,
        'radius': 65,
        'c': 'g'
    },
    {
        'simData': T2K_TEST,
        'radius': 75,
        'c': 'b'
    },
]
plotSimulationData(plotData)

In [ ]:
TESTDATA = runData(3011)

In [ ]:
_ = TESTDATA.plotAvalancheFits()
_ = TESTDATA.plotElectronEnergy()
_ = TESTDATA.plotDiffusion('electron')

In [ ]:
avalancheData = TESTDATA.getDataFrame('avalancheData')

IBNData = avalancheData['IBN']
aveIBN = TESTDATA.getCalcParameter('Average IBN')
aveIBNErr = TESTDATA.getCalcParameter('IBN Error')
aveIBNStdDev = TESTDATA.getCalcParameter('IBN StdDev')

low = IBNData.quantile(0.15865)
high = IBNData.quantile(0.84135)


plt.hist(IBNData, bins=IBNData.max())
plt.axvline(
    aveIBN,
    c='r', ls=':', label='Mean'
)
plt.axvspan(
    xmin=aveIBN-aveIBNErr, xmax=aveIBN+aveIBNErr,
    facecolor='r', alpha=.5
)

plt.axvspan(
    xmin=aveIBN-aveIBNStdDev, xmax=aveIBN+aveIBNStdDev,
    facecolor='g', alpha=.25
)

plt.axvspan(
    xmin=aveIBN-low, xmax=aveIBN+high,
    facecolor='c', alpha=.25
)

plt.show()

In [ ]:
#********************************************************************************#   
def plotIonizationLocation(runData, relativeScale=False):
    """
    TODO
    """
    #Get run parameters
    numRuns = runData.getRunParameter('Number of Avalanches')

    holeRadius = runData.getRunParameter('Hole Radius')
    padLength = runData.getRunParameter('Pad Length')
    padInner = np.sqrt(3)*padLength/2
    standoff = runData.getRunParameter('Grid Standoff')

    gain = runData.getCalcParameter('Trimmed Gain')

    #Get data and exclude primary electron
    electronData = runData.getDataFrame('electronData')
    lessInitial = electronData[electronData['Electron ID'] != 0].copy()

    #Find radius, and scale scale values relatve to standoff and hole radius
    lessInitial['Initial r'] = np.hypot(lessInitial['Initial x'], lessInitial['Initial y'])
    lessInitial['Relative x'] = lessInitial['Initial x']/holeRadius
    lessInitial['Relative y'] = lessInitial['Initial y']/holeRadius
    lessInitial['Relative z'] = lessInitial['Initial z']/standoff
    lessInitial['Relative r'] = np.hypot(lessInitial['Relative x'], lessInitial['Relative y'])

    #Use relative scale or not
    if relativeScale:
        plotID = 'Relative'
        fitLength=1
    else:
        plotID = 'Initial'
        fitLength=holeRadius

    #Do some maths and fits
    zMean = lessInitial[plotID+' z'].mean()
    rMean = lessInitial[plotID+' r'].mean()
    xStdDev = lessInitial[plotID+' x'].std()
    yStdDev = lessInitial[plotID+' y'].std()
    rayleighParam = rayleigh.fit(lessInitial[plotID+' r'], floc=0)
    riceB, riceLoc, riceScale = rice.fit(lessInitial[plotID+' r'], floc=0)
    
    xFit = np.linspace(0, fitLength, 100)
    raleighFit = rayleigh.pdf(xFit, *rayleighParam)
    riceFit = rice.pdf(xFit, riceB, loc=riceLoc, scale=riceScale)

    denom = 4 * xStdDev**2 * yStdDev**2
    A = (xStdDev**2 + yStdDev**2) / denom
    B = (xStdDev**2 - yStdDev**2) / denom
    hoytFit = (xFit / (xStdDev * yStdDev)) * np.exp(-A * xFit**2) * i0(B * xFit**2)

    #Set up labeling and data for plots
    if relativeScale:
        plotScaleZ = standoff 
        plotScaleR = holeRadius
        
        zLabel = 'z Position / Standoff'
        rLabel = 'r Position / Hole Radius'
    else:
        plotScaleZ = 1 
        plotScaleR = 1
        
        zLabel = 'z Position (um)'
        rLabel = 'r Position (um)'

    
    #Make Plots
    fig = plt.figure(figsize=(12, 4))
    fig.suptitle(f'Ionization Locations ({numRuns} Avalanches, Gain = {gain:.1f})')

    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    #Plot Data - do manually for scaling
    zCounts, zBins = np.histogram(lessInitial[plotID+' z'], bins=100, density=True)
    # Scale the density down so it matches the absolute values
    zCounts = zCounts/plotScaleZ 
    # Plot using stairs (or bar)
    ax1.stairs(
        zCounts, zBins, 
        orientation='horizontal', 
        fill=True
    )

    rCounts, rBins = np.histogram(lessInitial[plotID+' r'], bins=100, density=True)
    rCounts = rCounts/plotScaleR
    ax2.stairs(
        rCounts, rBins, 
        fill=True
    )

    #Add geometry/calculated values
    ax1.axhline(
        y=zMean, 
        c='r', ls='--', label=f'Mean = {zMean:.2f}'
    )
    ax1.axhline(
        y=0, 
        c='k', ls=':', label=f'Grid'
    )
    ax1.axhline(
        y=-standoff/plotScaleZ, 
        c='m', ls=':', label=f'Pad (Standoff = {standoff:.0f})'
    )
    
    ax2.axvline(
        x=rMean, 
        c='r', ls='--', label=f'Mean = {rMean:.2f}'
    )
    ax2.axvline(
        x=holeRadius/plotScaleR, 
        c='k', ls=':', label=f'Hole Radius ({holeRadius:.0f})'
    )
    ax2.axvline(
        x=padLength/plotScaleR, 
        c='m', ls='--', label=f'Pad Outer Radius ({padLength:.0f})'
    )
    ax2.axvline(
        x=padInner/plotScaleR, 
        c='m', ls=':', label=f'Pad Inner Radius ({padInner:.0f})'
    )

    #Plot fits
    ax2.plot(
        xFit, raleighFit/plotScaleR, 
        c='g', ls='-', label=f'Rayleigh Fit ({rayleighParam[1]:.2f})'
    )
    ax2.plot(
        xFit, hoytFit/plotScaleR, 
        c='g', ls='--', label=f'Hoyt Fit ({xStdDev:.2f}, {yStdDev:.2f})'
    )
    ax2.plot(
        xFit, riceFit/plotScaleR, 
         c='g', ls=':', label=f'Rice Fit ({riceB:.2f})'
    )

    ax1.set_title('Initial z (Excluding Primary)')
    ax2.set_title('Initial r (Excluding Primary)')

    ax1.set_xlabel('Probability Density (1/um)')
    ax1.set_ylabel(zLabel)
    ax2.set_xlabel(rLabel)
    ax2.set_ylabel('Probability Density (1/um)')
    
    ax1.legend()
    ax2.legend()

    plt.tight_layout()  

    return fig

_ = plotIonizationLocation(TESTDATA, False)
_ = plotIonizationLocation(TESTDATA, True)

In [ ]:
testPlot = {
    'FIMS (FR = 100)': list(range(3009, 3015)),
    'FIMS (FR = 75)': list(range(3015, 3021))
}
plotDataSets(testPlot, 'Grid Standoff', 'Trimmed Gain')
plotDataSets(testPlot, 'Grid Standoff', 'Theta')
#plotDataSets(testPlot, 'Grid Standoff', 'Average IBN')

In [ ]:
CF4_3 = gasData()
CF4_3.loadGasScans('myT2K', 'isobutane', 3)
_ = CF4_3.plotGasScan(25)
_ = CF4_3.plotGasEfficiencyScan()

In [ ]:
CF4_1 = gasData()
CF4_1.loadGasScans('myT2K', 'isobutane', 1)
CF4_3 = gasData()
CF4_3.loadGasScans('myT2K', 'isobutane', 3)
CF4_5 = gasData()
CF4_5.loadGasScans('myT2K', 'isobutane', 5)

In [ ]:
test1 = CF4_1.variabilitySWtest()
test3 = CF4_3.variabilitySWtest()
test5 = CF4_5.variabilitySWtest()

In [ ]:
allSW = [
    CF4_1.variabilitySWtest(),
    CF4_3.variabilitySWtest(),
    CF4_5.variabilitySWtest()
]
allW, allPval, allVariability = zip(*allSW)

allData = [
    CF4_1.gasEfficiencyData['variability'].tolist(),
    CF4_3.gasEfficiencyData['variability'].tolist(),
    CF4_5.gasEfficiencyData['variability'].tolist()
]
meanVariability = np.array(allData).mean()
shapiroW, shapiroPval = shapiro(allData)

concentrationCF4 = [1, 3, 5]

fig = plt.figure(figsize=(10, 4))
fig.suptitle(f'Shapiro-Wilk Tests:')
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.scatter(
    concentrationCF4, allVariability,
    marker='x', c='b', label='Mean Variability'
)
ax2.scatter(
    concentrationCF4, allW,
    marker='x', c='g', label='W Score'
)
ax2.scatter(
    concentrationCF4, allPval,
    marker='x', c='r', label='P Value'
)

ax1.axhline(y=meanVariability, c='b', ls=':', label=f'Global Mean = {meanVariability:.4f}')
ax2.axhline(y=shapiroW, c='g', ls=':', label=f'Global W Score = {shapiroW:.4f}')
ax2.axhline(y=shapiroPval, c='r', ls=':', label=f'Global P Value = {shapiroPval:.4f}')

ax1.set_title('Gain Variability')
ax2.set_title('Test Results')
ax1.set_ylabel(r'$1-n_{95}/\bar{n}$')

for inax in [ax1, ax2]:
    inax.grid()
    inax.legend()
    inax.set_ylim([0, 1])
    inax.set_xlabel('CF4 Concentration')

plt.tight_layout()
plt.show()
